# UniverSeg

- From the following paper: https://universeg.csail.mit.edu/
- From the git repo: https://github.com/JJGO/UniverSeg
- Form the example google colab: https://colab.research.google.com/drive/1TiNAgCehFdyHMJsS90V9ygUw0rLXdW0r?usp=sharing

## Setup

In [2]:
import math
import itertools

from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from universeg import universeg

model = universeg(pretrained=True)
_ = model.to(device)

/vol/bitbucket/az620/radiotherapy/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# Using visualisation code from tutorial in google colab
def visualize_tensors(tensors, col_wrap=8, col_names=None, title=None):
    M = len(tensors)
    N = len(next(iter(tensors.values())))

    cols = col_wrap
    rows = math.ceil(N/cols) * M

    d = 2.5
    fig, axes = plt.subplots(rows, cols, figsize=(d*cols, d*rows))
    if rows == 1:
      axes = axes.reshape(1, cols)

    for g, (grp, tensors) in enumerate(tensors.items()):
        for k, tensor in enumerate(tensors):
            col = k % cols
            row = g + M*(k//cols)
            x = tensor.detach().cpu().numpy().squeeze()
            ax = axes[row,col]
            if len(x.shape) == 2:
                ax.imshow(x,vmin=0, vmax=1, cmap='gray')
            else:
                ax.imshow(E.rearrange(x,'C H W -> H W C'))
            if col == 0:
                ax.set_ylabel(grp, fontsize=16)
            if col_names is not None and row == 0:
                ax.set_title(col_names[col])

    for i in range(rows):
        for j in range(cols):
            ax = axes[i,j]
            ax.grid(False)
            ax.set_xticks([])
            ax.set_yticks([])

    if title:
        plt.suptitle(title, fontsize=20)

    plt.tight_layout()

## Inference

Quoted from the tutorial, helpful text for setting up the context;

"""Given a new segmentation task (e.g. new biomedical domain, new image type, new region of interest, etc), most existing strategies involve training or fine-tuning a segmentation model (often a UNet-like CNN) that takes as input an image $x$ and outputs the segmentation map $y$.

This process works well in machine-learning labs, but is challenging in many applied settings, such as for scientists or clinical researchers who drive important scientific questions, but often lack the machine-learning expertiese and computational resources necessary.

UniverSeg enables users to tackle a new segmentation task without the need to train or fine-tune a model, removing the ML experience requirements and computational burden. The key idea is to have a *single* global model which adapts to a new segmentation task at inference. Given a new segmentation task, defined by a few example image-segmentation pairs $\mathcal{T} = \{x_n, y_n\}$, UniverSeg segments a new image $x$ by taking as input both $x$ and the task examples $\mathcal{T}$ and outputs the segmentation map $f(x, \mathcal{T}) = y$."""

### Load Data

In [4]:
import os

# Set up the path to the data.
data_path = '/vol/bitbucket/az620/radiotherapy/data'

if os.path.exists(data_path):
    print("The path exists.")
    print('The relevant contents are...')
    for x in sorted(os.listdir(data_path)): 
        if 'nnUNet' in x: print(f'   {x}')
else:
    print("The path does not exist.")


The path exists.
The relevant contents are...
   nnUNet_preprocessed
   nnUNet_raw
   nnUNet_results


In [5]:
# Each anatomical structure has a unique identifier.
id_anorectum = 'Dataset001_Anorectum'
id_bladder = 'Dataset002_Bladder'
id_ctvn = 'Dataset003_CTVn'
id_ctvp = 'Dataset004_CTVp'
id_parametrium = 'Dataset005_Parametrium'

In [6]:
import json

file_path = os.path.join(data_path, 'nnUNet_raw', id_anorectum, 'dataset.json')

with open(file_path, 'r') as file:
    data = json.load(file)

data

{'channel_names': {'0': 'CT'},
 'labels': {'background': 0, 'anorectum': 1},
 'numTraining': 100,
 'file_ending': '.nii.gz',
 'overwrite_image_reader_writer': 'SimpleITKIO'}

In [7]:
bladder_raw_data_x_path = os.path.join(data_path, 'nnUNet_raw', id_bladder, 'imagesTr')
bladder_label_y_path = os.path.join(data_path, 'nnUNet_raw', id_bladder, 'imagesTr')
bladder_output_path = os.path.join(data_path, 'UniverSeg', id_bladder, 'resultsTr')

os.makedirs(bladder_output_path, exist_ok=True)

In [8]:
import nibabel as nib

images = []
for file_path in os.listdir(bladder_raw_data_x_path):
    if file_path.endswith('.nii.gz'):
        image_path = os.path.join(bladder_raw_data_x_path, file_path)
        image = nib.load(image_path).get_fdata()
        images.append(image)
    break

# Convert the images to tensors
bladder_raw_data_x = torch.tensor(images).to(device)

/tmp/ipykernel_1291053/2070392723.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  bladder_raw_data_x = torch.tensor(images).to(device)


[1, 2, 3, 4, 79, 79, 79, 79]

### Visualise Some Examples